# Lesson 4: Docker Basics

**Module 5: Model Deployment**  
**Estimated Time**: 1 hour  
**Difficulty**: Intermediate

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Understand **Image** vs **Container**  
✅ Write an optimized `Dockerfile` for Python/ML  
✅ Learn about **Layer Caching** to speed up builds  
✅ Answer interview questions on containerization  

---

## 📚 Table of Contents

1. [The Concept: Shipping the Computer](#1-concept)
2. [The Dockerfile Anatomy](#2-dockerfile)
3. [Hands-On: Writing a Dockerfile](#3-hands-on)
4. [Interview Preparation](#4-interview-questions)

---

## 1. The Concept: Shipping the Computer

**Virtual Envs** isolate Python libraries.
**Docker** isolates the entire Operating System (Ubuntu, C++ drivers, etc.).

- **Image**: The Blueprint (Read-only).
- **Container**: The House (Running instance). You can spawn 100 containers from 1 image.

## 2. The Dockerfile Anatomy

We build images using instructions.

1. `FROM python:3.9-slim`: Base OS.
2. `WORKDIR /app`: `cd /app`.
3. `COPY requirements.txt .`: Move file in.
4. `RUN pip install ...`: Execute command.
5. `COPY . .`: Move rest of code.
6. `CMD ["uvicorn", ...]` : What to restart.

## 3. Hands-On: Writing a Dockerfile

We'll write a Dockerfile for our FastAPI app.

In [ ]:
dockerfile_content = """
# 1. Base Image (Slim is smaller, avoid Alpine for Python Data Science)
FROM python:3.9-slim

# 2. Set Env Vars (Prevent Pyc files, Unbuffered logs)
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# 3. Set Work Directory
WORKDIR /app

# 4. Install Dependencies (Copy ONLY requirements first for caching)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 5. Copy Code
COPY src/ src/

# 6. Create Non-Root User (Security Best Practice)
RUN adduser --disabled-password --gecos '' appuser && chown -R appuser /app
USER appuser

# 7. Run
CMD ["uvicorn", "src.main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

print(dockerfile_content)

print("\n--- Commands to Build & Run ---")
print("$ docker build -t my_ml_api:latest .")
print("$ docker run -p 8000:8000 my_ml_api:latest")

## 4. Interview Preparation

### Common Questions

#### Q1: "Why `COPY requirements.txt` before `COPY . .`?"
**Answer**: "To leverage **Layer Caching**. Docker caches each step. If I change my code (`main.py`), only the steps AFTER `COPY . .` re-run. If I copied everything at once, `pip install` would re-run every time I fixed a typo in my code, which is slow."

#### Q2: "Alpine vs Slim for Python?"
**Answer**: "I use **Slim** (Debian-based). **Alpine** (Musl-based) often breaks Python wheels (numpy/pandas) because they compile against glibc. This forces a compilation from source which creates huge images and takes forever."